In [1]:
import pandas as pd
import psycopg2
import json
from psycopg2.extras import execute_values

In [12]:
# Read the S&P 500 ticker metadata

url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

sp500_tickers = pd.read_html(url)[0]

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [15]:
# Keep relevant columns

sp500_tickers = sp500_tickers.drop(columns = ['GICS Sub-Industry', 'Headquarters Location', 'Date added', 'CIK', 'Founded'])

,Symbol,Security,GICS Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ACN,Accenture,Information Technology


In [20]:
# Rename column names 

sp500_tickers = sp500_tickers.rename(columns = {'Symbol':'ticker', 'Security':'company_name', 'GICS Sector':'industry'})

In [21]:
# Save to CSV

sp500_tickers.to_csv('raw_data/sp500_tickers.csv', index = False)

In [24]:
# Load the configuration file

file_path = 'config/config.txt'

# Open and load the JSON data
with open(file_path, 'r') as file:
    config = json.load(file)

# Now `data` is a Python dictionary (or list, depending on the file's structure)

In [25]:
# Pull the required configuration fields from the file

host = config['host']
dbname = config['dbName']
user = config['user']
password = config['password']
port = config['port']

In [27]:
# Define connection parameters
conn = psycopg2.connect( 
    host = host, # pgAdmin 4 > server properties > connection
    dbname = dbname, # pgAdmin 4 > database properties
    user = user, 
    password = password,
    port = port # pgAdmin 4 > server properties > connection
)

# Create a cursor object to run queries
cur = conn.cursor()

# Create a table in the database
create_table_script = '''
CREATE TABLE IF NOT EXISTS sp500_ticker_metadata(
    ticker TEXT,
    company_name TEXT,
    industry TEXT
);
'''

# Run the query & save the changes
cur.execute(create_table_script)
conn.commit()

In [29]:
# Convert dataframe rows into a list of tuples
rows = list(sp500_tickers.itertuples(index = False, name = None))
columns = ", ".join(sp500_tickers.columns)

# Insert the data from dataframe to the table in database
insert_script = f'''
INSERT INTO sp500_ticker_metadata ({columns})
VALUES %s
'''

# Using execute_values for multi-row insert
execute_values(cur, insert_script, rows)
conn.commit()

# Clean up
cur.close()
conn.close()

print(f'Appended {len(df)} rows to sp500 ticker metadata table')

Appended 503 rows to sp500 ticker metadata table
